![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)
# Migrating from HNSW to SVS-VAMANA

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/vector-search/06_hnsw_to_svs_migration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook demonstrates how to migrate existing HNSW vector indices to SVS-VAMANA for improved memory efficiency while maintaining search quality.

## What You'll Learn

- How to assess your current HNSW index for migration
- Step-by-step migration from HNSW to SVS-VAMANA
- Memory usage comparison and cost analysis
- Search quality validation between HNSW and SVS-VAMANA
- Performance benchmarking and recall comparison
- Migration decision framework for production systems

## Prerequisites

- Redis Stack 8.2.0+ with RediSearch 2.8.10+
- Existing HNSW index with substantial data (1000+ documents recommended)
- High-dimensional vectors (768+ dimensions for best compression benefits)

## HNSW vs SVS-VAMANA

**HNSW (Hierarchical Navigable Small World):**
- Excellent search quality and recall
- Fast query performance
- Higher memory usage (stores full-precision vectors)
- Good for applications prioritizing search quality

**SVS-VAMANA:**
- Competitive search quality with compression
- Significant memory savings (50-75% reduction)
- Built-in vector compression (LeanVec, quantization)
- Ideal for large-scale deployments with cost constraints

## 📦 Installation & Setup

This notebook requires **sentence-transformers** for generating embeddings and **Redis Stack** running in Docker.

**Requirements:**
- Redis Stack 8.2.0+ with RediSearch 2.8.10+
- sentence-transformers
- redisvl (should be available in your environment)

**🐳 Docker Setup (Required):**

Before running this notebook, make sure Redis Stack is running in Docker:

```bash
# Start Redis Stack with Docker
docker run -d --name redis-stack -p 6379:6379 -p 8001:8001 redis/redis-stack:latest
```

Or if you prefer using docker-compose, create a `docker-compose.yml` file:

```yaml
version: '3.8'
services:
  redis:
    image: redis/redis-stack:latest
    ports:
      - "6379:6379"
      - "8001:8001"
```

Then run: `docker-compose up -d`

**Manual Installation (if needed):**


**For Google Colab:** All dependencies will be installed automatically.

In [2]:
# # Install dependencies if needed
# import sys
# import subprocess

# def install_if_missing(package):
#     try:
#         __import__(package)
#     except ImportError:
#         print(f"Installing {package}...")
#         subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# # Check and install required packages
# install_if_missing("sentence-transformers")
# install_if_missing("redisvl")

# print("✅ All dependencies are ready!")

In [3]:
# Import required libraries
import os
import json
import numpy as np
import time
from typing import List, Dict, Any

# Redis and RedisVL imports
import redis
from redisvl.index import SearchIndex
from redisvl.query import VectorQuery
from redisvl.redis.utils import array_to_buffer, buffer_to_array
from redisvl.utils import CompressionAdvisor
from redisvl.redis.connection import supports_svs

# Configuration
REDIS_URL = "redis://localhost:6379"

print("📚 Libraries imported successfully!")

📚 Libraries imported successfully!


## Step 1: Verify Redis and SVS Support

First, let's ensure Redis Stack is running and supports SVS-VAMANA.

In [4]:
# Test Redis connection and SVS support
try:
    client = redis.Redis.from_url(REDIS_URL)
    client.ping()
    print("✅ Redis connection successful")
    
    # Check Redis version
    redis_info = client.info()
    redis_version = redis_info['redis_version']
    print(f"📊 Redis version: {redis_version}")
    
    # Check SVS support
    if supports_svs(client):
        print("✅ SVS-VAMANA supported")
    else:
        print("❌ SVS-VAMANA not supported")
        print("Please ensure you're using Redis Stack 8.2.0+ with RediSearch 2.8.10+")
        
except Exception as e:
    print(f"❌ Redis connection failed: {e}")
    print("Please ensure Redis Stack is running on localhost:6379")

✅ Redis connection successful
📊 Redis version: 8.2.2
✅ SVS-VAMANA supported


## Step 2: Load Sample Data

We'll use the movie dataset to demonstrate the migration process.

In [5]:
# Load the movies dataset
with open('resources/movies.json', 'r') as f:
    movies_data = json.load(f)

print(
    f"📽️ Loaded {len(movies_data)} movie records",
    f"Sample movie: {movies_data[0]['title']}",
    f"Genres available: {set(movie['genre'] for movie in movies_data)}",
    sep="\n"
)

# Configuration for demonstration  
dims = 1024  # sentence-transformers/all-roberta-large-v1 - 1024 dims
num_docs = len(movies_data)  # Use actual dataset size

print(
    f"\n🔧 Configuration:",
    f"Vector dimensions: {dims}",
    f"Dataset size: {num_docs} movie documents",
    sep="\n"
)

📽️ Loaded 20 movie records
Sample movie: Explosive Pursuit
Genres available: {'comedy', 'action'}

🔧 Configuration:
Vector dimensions: 1024
Dataset size: 20 movie documents


## Step 3: Create HNSW Index

First, we'll create an HNSW index with typical production settings.

In [6]:
# Create HNSW schema with production-like settings
hnsw_schema = {
    "index": {
        "name": "hnsw_demo_index",
        "prefix": "demo:hnsw:",
    },
    "fields": [
        {"name": "movie_id", "type": "tag"},
        {"name": "title", "type": "text"},
        {"name": "genre", "type": "tag"},
        {"name": "rating", "type": "numeric"},
        {"name": "description", "type": "text"},
        {
            "name": "embedding",
            "type": "vector",
            "attrs": {
                "dims": dims,
                "algorithm": "hnsw",
                "datatype": "float32",
                "distance_metric": "cosine",
                "m": 16,  # Number of bi-directional links for each node
                "ef_construction": 200,  # Size of dynamic candidate list
                "ef_runtime": 10  # Size of dynamic candidate list during search
            }
        }
    ]
}

print("Creating HNSW index with optimized settings...")
hnsw_index = SearchIndex.from_dict(hnsw_schema, redis_url=REDIS_URL)
hnsw_index.create(overwrite=True)
print(f"✅ Created HNSW index: {hnsw_index.name}")

# Display HNSW configuration
print(
    "\n🔧 HNSW Configuration:",
    f"M (connections per node): 16",
    f"EF Construction: 200",
    f"EF Runtime: 10",
    f"Distance metric: cosine",
    f"Data type: float32",
    sep="\n"
)

Creating HNSW index with optimized settings...
✅ Created HNSW index: hnsw_demo_index

🔧 HNSW Configuration:
M (connections per node): 16
EF Construction: 200
EF Runtime: 10
Distance metric: cosine
Data type: float32


## Step 4: Generate Embeddings and Load HNSW Index

Generate embeddings for movie descriptions and populate the HNSW index.

In [7]:
# Generate embeddings (synthetic for demonstration)
print("🔄 Generating embeddings for movie descriptions...")

try:
    # Try to use sentence-transformers if available
    from sentence_transformers import SentenceTransformer
    model = SentenceTransformer('all-MiniLM-L6-v2')  # 384 dimensions
    
    # Generate real embeddings
    descriptions = [movie['description'] for movie in movies_data]
    embeddings = model.encode(descriptions, convert_to_numpy=True)
    
    # Pad to 1024 dimensions for demonstration
    if embeddings.shape[1] < dims:
        padding = np.zeros((embeddings.shape[0], dims - embeddings.shape[1]))
        embeddings = np.concatenate([embeddings, padding], axis=1)
    
    print(f"✅ Generated real embeddings using SentenceTransformer")
    
except ImportError:
    # Fallback to synthetic embeddings
    print("📝 SentenceTransformer not available, generating synthetic embeddings...")
    
    np.random.seed(42)  # For reproducible results
    embeddings = []
    
    for i, movie in enumerate(movies_data):
        # Create a pseudo-semantic embedding based on movie content
        vector = np.random.random(dims).astype(np.float32)
        
        # Add some structure based on genre
        if movie['genre'] == 'action':
            vector[:50] += 0.3  # Action movies cluster
        else:  # comedy
            vector[50:100] += 0.3  # Comedy movies cluster
        
        # Normalize
        vector = vector / np.linalg.norm(vector)
        embeddings.append(vector)
    
    embeddings = np.array(embeddings)
    print(f"✅ Generated {len(embeddings)} synthetic embeddings")

print(f"📊 Embedding shape: {embeddings.shape}")

🔄 Generating embeddings for movie descriptions...
12:16:47 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: mps
12:16:47 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Generated real embeddings using SentenceTransformer
📊 Embedding shape: (20, 1024)


In [8]:
# Prepare data for loading into HNSW index
sample_data = []
for i, movie in enumerate(movies_data):
    sample_data.append({
        'movie_id': str(movie['id']),
        'title': movie['title'],
        'genre': movie['genre'],
        'rating': movie['rating'],
        'description': movie['description'],
        'embedding': array_to_buffer(embeddings[i].astype(np.float32), dtype='float32')
    })

print(f"📦 Prepared {len(sample_data)} documents for indexing")

📦 Prepared 20 documents for indexing


In [9]:
# Load data into HNSW index
print("📥 Loading data into HNSW index...")
batch_size = 100  # Process in batches

for i in range(0, len(sample_data), batch_size):
    batch = sample_data[i:i+batch_size]
    hnsw_index.load(batch)
    print(f"  Loaded {min(i+batch_size, len(sample_data))}/{len(sample_data)} documents")

# Wait for indexing to complete
print("⏳ Waiting for HNSW indexing to complete...")
time.sleep(5)  # HNSW indexing takes longer than FLAT

hnsw_info = hnsw_index.info()
print(
    f"\n✅ HNSW index loaded with {hnsw_info['num_docs']} documents",
    f"Index size: {hnsw_info.get('vector_index_sz_mb', 'N/A')} MB",
    f"Indexing time: ~5 seconds (HNSW graph construction)",
    sep="\n"
)

📥 Loading data into HNSW index...
  Loaded 20/20 documents
⏳ Waiting for HNSW indexing to complete...

✅ HNSW index loaded with 20 documents
Index size: 4.225975036621094 MB
Indexing time: ~5 seconds (HNSW graph construction)


## Step 5: Get Compression Recommendation

Use the CompressionAdvisor to get optimal SVS-VAMANA settings for our data.

In [10]:
# Get compression recommendation
print("🔍 Analyzing data for optimal compression settings...")

# Get recommendations for different priorities
memory_config = CompressionAdvisor.recommend(dims=dims, priority="memory")
balanced_config = CompressionAdvisor.recommend(dims=dims, priority="balanced")
performance_config = CompressionAdvisor.recommend(dims=dims, priority="performance")

print(
    "\n📊 Compression Recommendations:",
    "",
    "🗜️ Memory Priority:",
    f"  Algorithm: {memory_config['algorithm']}",
    f"  Compression: {memory_config.get('compression', 'None')}",
    f"  Datatype: {memory_config['datatype']}",
    f"  Dimensions: {dims} → {memory_config.get('reduce', dims)}",
    "",
    "⚖️ Balanced Priority:",
    f"  Algorithm: {balanced_config['algorithm']}",
    f"  Compression: {balanced_config.get('compression', 'None')}",
    f"  Datatype: {balanced_config['datatype']}",
    f"  Dimensions: {dims} → {balanced_config.get('reduce', dims)}",
    "",
    "⚡ Performance Priority:",
    f"  Algorithm: {performance_config['algorithm']}",
    f"  Compression: {performance_config.get('compression', 'None')}",
    f"  Datatype: {performance_config['datatype']}",
    f"  Dimensions: {dims} → {performance_config.get('reduce', dims)}",
    sep="\n"
)

# Select configuration (using memory priority for maximum savings)
selected_config = memory_config
target_dims = selected_config.get('reduce', dims)
target_dtype = selected_config['datatype']

print(
    f"\n✅ Selected configuration: Memory Priority",
    f"Expected memory reduction: ~{((dims - target_dims) / dims * 100):.1f}% from dimension reduction",
    f"Additional savings from {selected_config['datatype']} compression",
    sep="\n"
)

🔍 Analyzing data for optimal compression settings...

📊 Compression Recommendations:

🗜️ Memory Priority:
  Algorithm: svs-vamana
  Compression: LeanVec4x8
  Datatype: float16
  Dimensions: 1024 → 512

⚖️ Balanced Priority:
  Algorithm: svs-vamana
  Compression: LeanVec4x8
  Datatype: float16
  Dimensions: 1024 → 512

⚡ Performance Priority:
  Algorithm: svs-vamana
  Compression: LeanVec4x8
  Datatype: float16
  Dimensions: 1024 → 512

✅ Selected configuration: Memory Priority
Expected memory reduction: ~50.0% from dimension reduction
Additional savings from float16 compression


## Step 6: Create SVS-VAMANA Index

Create the SVS-VAMANA index with the recommended compression settings.

In [11]:
# Create SVS-VAMANA schema with compression
svs_schema = {
    "index": {
        "name": "svs_demo_index",
        "prefix": "demo:svs:",
    },
    "fields": [
        {"name": "movie_id", "type": "tag"},
        {"name": "title", "type": "text"},
        {"name": "genre", "type": "tag"},
        {"name": "rating", "type": "numeric"},
        {"name": "description", "type": "text"},
        {
            "name": "embedding",
            "type": "vector",
            "attrs": {
                "dims": target_dims,  # Use reduced dimensions (512)
                "algorithm": "svs-vamana",
                "datatype": selected_config['datatype'],
                "distance_metric": "cosine"
                # Note: Don't include the full selected_config to avoid dims/reduce conflict
            }
        }
    ]
}

print("Creating SVS-VAMANA index with compression...")
svs_index = SearchIndex.from_dict(svs_schema, redis_url=REDIS_URL)
svs_index.create(overwrite=True)
print(
    f"✅ Created SVS-VAMANA index: {svs_index.name}",
    f"Compression: {selected_config.get('compression', 'None')}",
    f"Datatype: {selected_config['datatype']}",
    f"Dimensions: {dims} → {target_dims}",
    sep="\n"
)

Creating SVS-VAMANA index with compression...
✅ Created SVS-VAMANA index: svs_demo_index
Compression: LeanVec4x8
Datatype: float16
Dimensions: 1024 → 512


## Step 7: Migrate Data from HNSW to SVS-VAMANA

Extract data from the HNSW index and migrate it to SVS-VAMANA with compression.

In [12]:
# Extract data from HNSW index
print("🔄 Extracting data from HNSW index...")

client = redis.Redis.from_url(REDIS_URL)
keys = client.keys("demo:hnsw:*")
print(f"Found {len(keys)} documents to migrate")

# Process and transform data for SVS index
svs_data = []

for key in keys:
    doc_data = client.hgetall(key)
    
    if b'embedding' in doc_data:
        # Extract original vector from HNSW index
        original_vector = np.array(buffer_to_array(doc_data[b'embedding'], dtype='float32'))
        
        # Apply dimensionality reduction if needed (LeanVec)
        if target_dims < dims:
            vector = original_vector[:target_dims]
        else:
            vector = original_vector
        
        # Convert to target datatype
        if target_dtype == 'float16':
            vector = vector.astype(np.float16)
        
        svs_data.append({
            "movie_id": doc_data[b'movie_id'].decode(),
            "title": doc_data[b'title'].decode(),
            "genre": doc_data[b'genre'].decode(),
            "rating": int(doc_data[b'rating'].decode()),
            "description": doc_data[b'description'].decode(),
            "embedding": array_to_buffer(vector, dtype=target_dtype)
        })

print(f"Prepared {len(svs_data)} documents for migration")

🔄 Extracting data from HNSW index...
Found 20 documents to migrate
Prepared 20 documents for migration


In [13]:
# Load data into SVS index
print("📥 Loading data into SVS-VAMANA index...")
batch_size = 100  # Define batch size for migration

if len(svs_data) > 0:
    for i in range(0, len(svs_data), batch_size):
        batch = svs_data[i:i+batch_size]
        svs_index.load(batch)
        print(f"  Migrated {min(i+batch_size, len(svs_data))}/{len(svs_data)} documents")

    # Wait for indexing to complete
    print("⏳ Waiting for SVS-VAMANA indexing to complete...")
    time.sleep(5)

    svs_info = svs_index.info()
    print(
        f"\n✅ Migration complete! SVS index has {svs_info['num_docs']} documents",
        f"Index size: {svs_info.get('vector_index_sz_mb', 'N/A')} MB",
        sep="\n"
    )
else:
    print("⚠️  No data to migrate. Make sure the HNSW index was populated first.")
    print("   Run the previous cells to load data into the HNSW index.")
    svs_info = svs_index.info()

📥 Loading data into SVS-VAMANA index...
  Migrated 20/20 documents
⏳ Waiting for SVS-VAMANA indexing to complete...

✅ Migration complete! SVS index has 20 documents
Index size: 1.017791748046875 MB


## Step 8: Compare Memory Usage

Analyze the memory savings achieved through the HNSW to SVS-VAMANA migration.

In [14]:
# Helper function to extract memory info
def get_memory_mb(index_info):
    """Extract memory usage in MB from index info"""
    memory = index_info.get('vector_index_sz_mb', 0)
    if isinstance(memory, str):
        try:
            return float(memory)
        except ValueError:
            return 0.0
    return float(memory)

# Get memory usage
hnsw_memory = get_memory_mb(hnsw_info)
svs_memory = get_memory_mb(svs_info)

print(
    "📊 Memory Usage Comparison",
    "=" * 40,
    f"Original HNSW index:    {hnsw_memory:.2f} MB",
    f"SVS-VAMANA index:       {svs_memory:.2f} MB",
    "",
    sep="\n"
)

if hnsw_memory > 0:
    if svs_memory > 0:
        savings = ((hnsw_memory - svs_memory) / hnsw_memory) * 100
        print(
            f"💰 Memory savings: {savings:.1f}%",
            f"Absolute reduction: {hnsw_memory - svs_memory:.2f} MB",
            sep="\n"
        )
    else:
        print("⏳ SVS index still indexing - memory comparison pending")
        
    # Cost analysis
    print("\n💵 Cost Impact Analysis:")
    cost_per_gb_hour = 0.10  # Example cloud pricing
    hours_per_month = 24 * 30
    
    hnsw_monthly_cost = (hnsw_memory / 1024) * cost_per_gb_hour * hours_per_month
    if svs_memory > 0:
        svs_monthly_cost = (svs_memory / 1024) * cost_per_gb_hour * hours_per_month
        monthly_savings = hnsw_monthly_cost - svs_monthly_cost
        print(
            f"Monthly cost reduction: ${monthly_savings:.2f}",
            f"Annual cost reduction: ${monthly_savings * 12:.2f}",
            sep="\n"
        )
    else:
        print(
            f"Current monthly cost: ${hnsw_monthly_cost:.2f}",
            "Projected savings: Available after indexing completes",
            sep="\n"
        )
else:
    print("⚠️  Memory information not available")

📊 Memory Usage Comparison
Original HNSW index:    4.23 MB
SVS-VAMANA index:       1.02 MB

💰 Memory savings: 75.9%
Absolute reduction: 3.21 MB

💵 Cost Impact Analysis:
Monthly cost reduction: $0.23
Annual cost reduction: $2.71


## Step 9: Validate Search Quality

Compare search quality between HNSW and SVS-VAMANA to ensure the migration maintains acceptable recall.

In [15]:
# Generate test queries
print("🔍 Generating test queries for quality validation...")

np.random.seed(123)  # For reproducible test queries
num_test_queries = 10
test_queries = []

for i in range(num_test_queries):
    # Create test query vectors
    query_vec = np.random.random(dims).astype(np.float32)
    query_vec = query_vec / np.linalg.norm(query_vec)  # Normalize
    test_queries.append(query_vec)

print(f"Generated {len(test_queries)} test queries")

🔍 Generating test queries for quality validation...
Generated 10 test queries


In [16]:
# Test HNSW search quality
print("🔍 Testing HNSW search quality...")

hnsw_results = []
hnsw_start = time.time()

for query_vec in test_queries:
    query = VectorQuery(
        vector=query_vec,
        vector_field_name="embedding",
        return_fields=["movie_id", "title", "genre"],
        dtype="float32",
        num_results=10
    )
    results = hnsw_index.query(query)
    hnsw_results.append([doc["movie_id"] for doc in results])

hnsw_time = time.time() - hnsw_start
print(f"HNSW search completed in {hnsw_time:.3f} seconds")

🔍 Testing HNSW search quality...
HNSW search completed in 0.011 seconds


In [17]:
# Test SVS-VAMANA search quality
print("🔍 Testing SVS-VAMANA search quality...")

svs_results = []
svs_start = time.time()

for i, query_vec in enumerate(test_queries):
    # Adjust query vector for SVS index (handle dimensionality reduction)
    if target_dims < dims:
        svs_query_vec = query_vec[:target_dims]
    else:
        svs_query_vec = query_vec
    
    if target_dtype == 'float16':
        svs_query_vec = svs_query_vec.astype(np.float16)
    
    query = VectorQuery(
        vector=svs_query_vec,
        vector_field_name="embedding",
        return_fields=["movie_id", "title", "genre"],
        dtype=target_dtype,
        num_results=10
    )
    results = svs_index.query(query)
    svs_results.append([doc["movie_id"] for doc in results])

svs_time = time.time() - svs_start
print(f"SVS-VAMANA search completed in {svs_time:.3f} seconds")

🔍 Testing SVS-VAMANA search quality...
SVS-VAMANA search completed in 0.008 seconds


In [18]:
# Calculate recall and performance metrics
def calculate_recall(reference_results, test_results, k=10):
    """Calculate recall@k between two result sets"""
    if not reference_results or not test_results:
        return 0.0
    
    total_recall = 0.0
    for ref, test in zip(reference_results, test_results):
        ref_set = set(ref[:k])
        test_set = set(test[:k])
        if len(ref_set) > 0:
            recall = len(ref_set.intersection(test_set)) / len(ref_set)
            total_recall += recall
    
    return total_recall / len(reference_results)

# Calculate metrics
recall_at_5 = calculate_recall(hnsw_results, svs_results, k=5)
recall_at_10 = calculate_recall(hnsw_results, svs_results, k=10)

print(
    "📊 Search Quality Comparison",
    "=" * 40,
    f"Recall@5:  {recall_at_5:.3f} ({recall_at_5*100:.1f}%)",
    f"Recall@10: {recall_at_10:.3f} ({recall_at_10*100:.1f}%)",
    "",
    "⏱️ Performance Comparison:",
    f"HNSW query time:     {hnsw_time:.3f}s ({hnsw_time/num_test_queries*1000:.1f}ms per query)",
    f"SVS-VAMANA query time: {svs_time:.3f}s ({svs_time/num_test_queries*1000:.1f}ms per query)",
    f"Speed difference:    {((hnsw_time - svs_time) / hnsw_time * 100):+.1f}%",
    sep="\n"
)

# Quality assessment
if recall_at_10 >= 0.95:
    quality_assessment = "🟢 Excellent - Minimal quality loss"
elif recall_at_10 >= 0.90:
    quality_assessment = "🟡 Good - Acceptable quality for most applications"
elif recall_at_10 >= 0.80:
    quality_assessment = "🟠 Fair - Consider if quality requirements are flexible"
else:
    quality_assessment = "🔴 Poor - Migration not recommended"

print(f"\n🎯 Quality Assessment: {quality_assessment}")

📊 Search Quality Comparison
Recall@5:  1.000 (100.0%)
Recall@10: 0.990 (99.0%)

⏱️ Performance Comparison:
HNSW query time:     0.011s (1.1ms per query)
SVS-VAMANA query time: 0.008s (0.8ms per query)
Speed difference:    +23.0%

🎯 Quality Assessment: 🟢 Excellent - Minimal quality loss


## Step 10: Migration Decision Framework

Based on the analysis, determine if migration is recommended.

In [19]:
# Migration decision logic
memory_savings_threshold = 20  # Minimum 20% memory savings
recall_threshold = 0.85  # Minimum 85% recall@10

memory_savings_pct = ((hnsw_memory - svs_memory) / hnsw_memory * 100) if hnsw_memory > 0 and svs_memory > 0 else 0
meets_memory_threshold = memory_savings_pct >= memory_savings_threshold
meets_quality_threshold = recall_at_10 >= recall_threshold

print(
    "🤔 Migration Decision Analysis",
    "=" * 40,
    "",
    "📊 Criteria Evaluation:",
    f"Memory savings: {memory_savings_pct:.1f}% {'✅' if meets_memory_threshold else '❌'} (threshold: {memory_savings_threshold}%)",
    f"Search quality: {recall_at_10:.3f} {'✅' if meets_quality_threshold else '❌'} (threshold: {recall_threshold})",
    "",
    sep="\n"
)

if meets_memory_threshold and meets_quality_threshold:
    recommendation = "🟢 RECOMMENDED"
    reasoning = "Migration provides significant memory savings while maintaining good search quality."
elif meets_memory_threshold and not meets_quality_threshold:
    recommendation = "🟡 CONDITIONAL"
    reasoning = "Good memory savings but reduced search quality. Consider if your application can tolerate lower recall."
elif not meets_memory_threshold and meets_quality_threshold:
    recommendation = "🟠 LIMITED BENEFIT"
    reasoning = "Search quality is maintained but memory savings are minimal. Migration may not be worth the effort."
else:
    recommendation = "🔴 NOT RECOMMENDED"
    reasoning = "Insufficient memory savings and/or poor search quality. Consider alternative optimization strategies."

print(
    f"🎯 Migration Recommendation: {recommendation}",
    f"💭 Reasoning: {reasoning}",
    sep="\n"
)

🤔 Migration Decision Analysis

📊 Criteria Evaluation:
Memory savings: 75.9% ✅ (threshold: 20%)
Search quality: 0.990 ✅ (threshold: 0.85)

🎯 Migration Recommendation: 🟢 RECOMMENDED
💭 Reasoning: Migration provides significant memory savings while maintaining good search quality.


## Step 11: Production Migration Checklist

If migration is recommended, follow this checklist for production deployment.

In [20]:
print(
    "📋 HNSW to SVS-VAMANA Migration Checklist",
    "=" * 50,
    "\nPRE-MIGRATION:",
    "□ Backup existing HNSW index data",
    "□ Test migration on staging environment",
    "□ Validate search quality with real queries",
    "□ Measure baseline HNSW performance metrics",
    "□ Plan rollback strategy",
    "□ Document current HNSW parameters (M, EF_construction, EF_runtime)",
    "\nMIGRATION:",
    "□ Create SVS-VAMANA index with tested configuration",
    "□ Migrate data in batches during low-traffic periods",
    "□ Monitor memory usage and indexing progress",
    "□ Validate data integrity after migration",
    "□ Test search functionality thoroughly",
    "□ Compare recall metrics with baseline",
    "\nPOST-MIGRATION:",
    "□ Monitor search performance and quality",
    "□ Track memory usage and cost savings",
    "□ Update application configuration",
    "□ Document new SVS-VAMANA settings",
    "□ Clean up old HNSW index after validation period",
    "□ Update monitoring and alerting thresholds",
    "\n💡 HNSW-SPECIFIC TIPS:",
    "• HNSW indices are more complex to rebuild than FLAT",
    "• Consider the impact on applications using EF_runtime tuning",
    "• SVS-VAMANA may have different optimal query parameters",
    "• Test with your specific HNSW configuration (M, EF values)",
    "• Monitor for 48-72 hours before removing HNSW index",
    "• Keep compression settings documented for future reference",
    sep="\n"
)

📋 HNSW to SVS-VAMANA Migration Checklist

PRE-MIGRATION:
□ Backup existing HNSW index data
□ Test migration on staging environment
□ Validate search quality with real queries
□ Measure baseline HNSW performance metrics
□ Plan rollback strategy
□ Document current HNSW parameters (M, EF_construction, EF_runtime)

MIGRATION:
□ Create SVS-VAMANA index with tested configuration
□ Migrate data in batches during low-traffic periods
□ Monitor memory usage and indexing progress
□ Validate data integrity after migration
□ Test search functionality thoroughly
□ Compare recall metrics with baseline

POST-MIGRATION:
□ Monitor search performance and quality
□ Track memory usage and cost savings
□ Update application configuration
□ Document new SVS-VAMANA settings
□ Clean up old HNSW index after validation period
□ Update monitoring and alerting thresholds

💡 HNSW-SPECIFIC TIPS:
• HNSW indices are more complex to rebuild than FLAT
• Consider the impact on applications using EF_runtime tuning
• SVS-VA

## Step 12: Cleanup

Clean up the demonstration indices.

In [21]:
print("🧹 Cleaning up demonstration indices...")

# Clean up HNSW index
try:
    hnsw_index.delete(drop=True)
    print("✅ Deleted HNSW demonstration index")
except Exception as e:
    print(f"⚠️  Failed to delete HNSW index: {e}")

# Clean up SVS index
try:
    svs_index.delete(drop=True)
    print("✅ Deleted SVS-VAMANA demonstration index")
except Exception as e:
    print(f"⚠️  Failed to delete SVS index: {e}")

print(
    "\n🎉 HNSW to SVS-VAMANA migration demonstration complete!",
    "\nNext steps:",
    "1. Apply learnings to your production HNSW indices",
    "2. Test with your actual query patterns and data",
    "3. Monitor performance in your environment",
    "4. Consider gradual rollout strategy",
    "5. Evaluate impact on applications using HNSW-specific features",
    sep="\n"
)

🧹 Cleaning up demonstration indices...
✅ Deleted HNSW demonstration index
✅ Deleted SVS-VAMANA demonstration index

🎉 HNSW to SVS-VAMANA migration demonstration complete!

Next steps:
1. Apply learnings to your production HNSW indices
2. Test with your actual query patterns and data
3. Monitor performance in your environment
4. Consider gradual rollout strategy
5. Evaluate impact on applications using HNSW-specific features
